In [ ]:
"""
Cell For Papermill Parameters
"""

images_to_use = 25
x_crop = 50
y_crop = 50
folder_path = "../../prostate_cancer_data/"

In [ ]:
import pydicom as dicom
import os
import cv2
import pandas as pd
import math
import glob
from pathlib import Path

In [ ]:
def process_folder(folder_path):
    
    """
    process_folder processes DCOM images in the folder
    
    :param folder_path: folder to process
    :return: jpg_list,dcm_values
    """ 
    
    jpg_list = []
    dcm_values = []
        
    fileList = glob.glob(folder_path + '**/*dcm*' , recursive=True)
    
    for image in fileList:
        
        if image.endswith(".dcm"):
            ds = dicom.dcmread(image)
                    
            for i in range(0,(len(ds.pixel_array)-1)):
                jpg_image_name = image+str(i).zfill(8)+".jpg"
                cv2.imwrite(jpg_image_name, ds.pixel_array[i])                
                jpg_list.append(jpg_image_name)
                                                               
    return jpg_list

In [ ]:
def crop_image(folder_path,x_crop,y_crop):
    
    """
    crop_image crops the image
    
    :param folder_path: folder to process
    :param x_crop: pixel crop x
    :param y_crop: pixel crop y
    """ 
    
    fileList = glob.glob(folder_path + '**/*jpg*')
    
    # Iterate over images
    for filePath in fileList:

        img = cv2.imread(filePath)
        image_shape_x = img.shape[0]
        image_shape_y = img.shape[1]
                        
        crop_img = img[(0+y_crop):(image_shape_y-y_crop), (0+x_crop):(image_shape_x-x_crop)]
        cv2.imwrite(filePath, crop_img)
        

In [ ]:
def remove_jpg(folder_path):
    
    """
    remove_jpg removes jpg from folder
    
    :param folder_path: folder to process
    """ 
    
    # ==========================================================================================
    # Remove old Final Files
    # ==========================================================================================
    fileList = glob.glob(folder_path + '**/*Rendering*/*jpg*' , recursive=True)
    
    # Iterate over the list of filepaths & remove each file.
    for filePath in fileList:
        try:
            os.remove(filePath)            
        except:
            pass

In [ ]:
def make_patient_image(folder_path,images_to_use):

    """
    make_patient_image creates images
    
    :param folder_path: folder to process
    :param images_to_use: images to use
    """ 
    
    # ==========================================================================================
    # List all Images
    # ==========================================================================================
    image_list = []    
    images_path = glob.glob(folder_path + '**/*jpg*')
                        
    for image in images_path:
        if image.endswith(".jpg"):
            image_list.append(image)            
                    
    #sort images
    image_list.sort()
    
    image_cnt = len(image_list)
    
    image_ignore = math.floor((image_cnt- images_to_use)/2)
            
    #remove first x images
    image_list = image_list[image_ignore:]
    
    #remove last x images
    image_list = image_list[:len(image_list)-image_ignore]
    
    image_list = image_list[:images_to_use]
            
    image_cnt = len(image_list)
    print(image_cnt)
    
    
    if image_cnt == 25:
        dims = 5
    elif image_cnt == 16:
        dims = 4
    elif image_cnt == 4:
        dims = 2
    elif image_cnt < 4:
        dims = 1        
    else:
        dims = math.ceil((math.sqrt(image_cnt)))   
        
    # ==========================================================================================
    # Looping Over Images to Concatenate and create Vertical
    # ==========================================================================================
    for coll_cnt in range(0,dims):

        final_image = os.path.join(folder_path, "US_fnlV" + str(coll_cnt)+".jpg")
        zero_flag = 0
        
        #print("======================================")
        
        for x in range(1,(dims)):        

            image_number = (x - 1) + (coll_cnt * dims)

            if image_number >= (image_cnt-1):
                # set image file names
                img1 = final_image
                img2 = image_list[(0)]

            elif zero_flag == 0:
                # set image file names
                img1 = image_list[image_number]
                img2 = image_list[(image_number+1)]            
                zero_flag = 1
            else:
                # set image file names
                img1 = final_image
                img2 = image_list[(image_number+1)]
                
            # read the images           
            img1 = cv2.imread(img1)
            img2 = cv2.imread(img2)
            
            try:
                im_v = cv2.vconcat([img1, img2])   
                cv2.imwrite(final_image, im_v)        
            except:
                pass

            
    # ==========================================================================================
    # Looping Over finalV images to concatenate into final image
    # ==========================================================================================    
    
    final_image = os.path.join(folder_path, "US_fnl.jpg")
    image_list = glob.glob(folder_path + '*US_fnlV*')
    image_cnt = len(image_list) - 1
    zero_flag = 0
    image_list.sort()
        
    for x in range(0,image_cnt):

        if zero_flag == 0:
            # set image file names
            img1 = image_list[x]
            img2 = image_list[(x+1)]            
            zero_flag = 1
        else:
            # set image file names
            img1 = final_image
            img2 = image_list[(x+1)]
            
        # read the images
        img1 = cv2.imread(img1)
        img2 = cv2.imread(img2)

        im_v = cv2.hconcat([img1, img2])   
        cv2.imwrite(final_image, im_v)        


In [ ]:
fileList = glob.glob(folder_path + '**/*Rendering*' , recursive=True)

dcm_metadata = []

print(len(fileList))
counter = 0

# Remove all JPG
remove_jpg(folder_path)

# Sort List
fileList.sort()

for folder in fileList:
    print("=====================================================================================")
    print(counter)
    print(folder)
            
    # Convert dicom to JPEG
    jpg_list = process_folder(folder+"/")
        
    # Crop Image
    crop_image(folder+"/",x_crop,y_crop)
    
    # Make collage
    make_patient_image(folder+"/",images_to_use)
    
    counter = counter + 1